In [24]:
import pendulum

from datetime import datetime, timedelta

TICKERS = ['FB', 'GOOG']
API_KEY="W2KCGVWHV34K0OZT74Q2ZDZ751FGRXUT"
# Set arguments
us_east_tz = pendulum.timezone('America/Toronto')
default_args = {
    'owner': 'vardhan',
    'start_date': datetime(2022, 1, 7, 7, 30, tzinfo=us_east_tz),
    'retries': 1,
    'retry_delay': timedelta(minutes=1)
}

# method for date to epoch 
def date_to_epoch(date):
    return int(date.timestamp())

def extract_historic_data_from_tda(ticker, ti):
    import json 
    import requests

    # Configure dates
    # start_date 5 years back 
    TIMEDELTA = timedelta(days=365)
    # start_date = datetime.today(tzinfo=us_east_tz)
    end_date = datetime.utcnow().replace(tzinfo=us_east_tz)
    start_date = end_date - TIMEDELTA

    # Configure request
    headers = {
        'Authorization': '',
    }

    params = (
        ('apikey', API_KEY),
        ('startDate', date_to_epoch(start_date)),
    )
    url = "https://api.tdameritrade.com/v1/marketdata/"+ ticker +"/pricehistory".format(ticker=ticker)
    # Get data
    response = requests.get(
        url,
        headers=headers,
        params=params
    )
    print(response)
    data = json.loads(response.content)
    # Push XCOM
    ti.xcom_push(key='raw_data', value=data)

def transform_historic_data(ticker, ti):
    
    # Import modules
    import pandas as pd

    # Get data
    data = ti.xcom_pull(key='raw_data', task_ids=["extract_historic_data_from_tda"])[0]

    # Define Columns
    columns = [
        'symbol',
        'date',
        'high_price',
        'low_price',
        'open_price',
        'close_price',
        'total_volume'
    ]

    # extract data
    extract_data = []
    for item in data['candles']:
        extract_data.append([
            item['symbol']=ticker,
            item['datetime'],
            item['high'],
            item['low'],
            item['open'],
            item['close'],
            item['volume']
        ])
    
    # Convert to dataframe
    df = pd.DataFrame(extract_data, columns=columns)
    print(df)

    # Convert floats
    def conv_num(x):
        return pd.to_numeric(x.astype(str).str.replace('NaN|nan', '', regex=True))
    
    for col in ["high_price", "low_price", "open_price", "close_price", "total_volume"]:
        df[col] = conv_num(df[col])
    
    # Convert strings
    def conv_str(x):
        return x.astype(str)

    for col in ['symbol']:
        df[col] = conv_str(df[col])

    # Convert integers
    def conv_int(x):
        return x.astype(int)

    for col in ['date']:
        df[col] = conv_int(df[col])

    # Fill missing values
    df = df.fillna(-99)
    ti.xcom_push(key='transformed_historic_data', value=df.to_dict('records'))


extract_historic_data_from_tda('GOOG')
transform_historic_data('GOOG')

this is the end_date that is today 2022-07-26 02:20:15.975354-04:00
this is the start_date that is 5 years ago 2021-07-26 02:20:15.975354-04:00
1627280415
1658816415
this is the requests params
this is the url https://api.tdameritrade.com/v1/marketdata/GOOG/pricehistory
<Response [200]>
{'candles': [{'open': 116.95, 'high': 116.95, 'low': 116.95, 'close': 116.95, 'volume': 4000, 'datetime': 1654691040000}, {'open': 117.2295, 'high': 117.2295, 'low': 117.2295, 'close': 117.2295, 'volume': 40000, 'datetime': 1654692120000}, {'open': 116.882, 'high': 117.28525, 'low': 116.69675, 'close': 117.2235, 'volume': 281360, 'datetime': 1654695000000}, {'open': 117.0095, 'high': 117.0095, 'low': 116.792, 'close': 116.8545, 'volume': 36100, 'datetime': 1654695060000}, {'open': 116.8655, 'high': 116.89823, 'low': 116.719775, 'close': 116.89175, 'volume': 46000, 'datetime': 1654695120000}, {'open': 116.8465, 'high': 117.021, 'low': 116.807, 'close': 116.807, 'volume': 46440, 'datetime': 1654695180000}